# AstraForge Toolkit - DeepAgent Ralph Mode (Notebook)

Run a Ralph-style autonomous loop using the DeepAgents Python API with the AstraForge sandbox backend.
Each iteration starts with fresh agent context, while `/workspace` acts as memory between runs.

Prereqs:
- Backend running at `http://localhost:8001` (e.g., `make backend-serve`)
- An API key (create via the in-app API Keys screen or `/api/api-keys/`)
- Package installed (`pip install astraforge-toolkit`)
- A model provider (this example uses Ollama via `langchain-ollama`)


## Notebook walk-through

1. Install the toolkit, dotenv helpers, DeepAgents, and the Ollama runtime so the notebook can reach your local model provider.
2. Load `.env`, derive `BASE_URL`/`API_KEY`, and open a sandbox session that backs every iteration; the session ID is reprinted for continuity.
3. Create a `ChatOllama` LLM, wire it into `create_deep_agent`, and configure `SandboxBackend` plus the shell tool so DeepAgent can read/write `/workspace`.
4. `run_ralph_loop` uses fresh thread IDs while keeping `/workspace` as cross-iteration memory; it prints each iteration summary and returns the raw message history for inspection.
5. After iterating, the notebook flattens every message for richer inspection and shows how to stop the sandbox session when you finish.


### Visual flow overview

- Prepare the Ollama LLM, sandbox backend, and tools before kicking off Ralph iterations.
- Each loop prints an iteration header and captures the raw history so you can compare progress.
- The flattened messages cell renders an HTML view of every utterance for easier validation.
- Stop the sandbox session manually when you are done to keep `/workspace` tidy.


In [14]:
# Install dependencies (run from the examples/ folder)
%pip install astraforge-toolkit python-dotenv deepagents langchain-ollama --quiet


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import os

BASE_URL = os.getenv("ASTRA_FORGE_API_URL")
API_KEY = os.getenv("ASTRA_FORGE_API_KEY")  # for local setup go to http://localhost:5174/app/api-keys

if not API_KEY:
    raise RuntimeError("Set ASTRA_FORGE_API_KEY in your environment")

In [17]:
from astraforge_toolkit import DeepAgentClient

client = DeepAgentClient(base_url=BASE_URL, api_key=API_KEY)
sandbox_session = client.create_sandbox_session()
SANDBOX_SESSION_ID = sandbox_session.session_id
print(f"Sandbox session: {SANDBOX_SESSION_ID} (workspace: {sandbox_session.workspace_path})")

Sandbox session: d1a49255-2684-4650-a5e0-384a064bac53 (workspace: /workspace)


In [18]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="devstral-small-2:24b",
    validate_model_on_init=True,
    temperature=0,
    model_kwargs={"think": "high"},
)

In [19]:
from deepagents import create_deep_agent
from astraforge_toolkit import SandboxBackend, sandbox_shell

def backend_factory(rt):
    return SandboxBackend(
        rt,
        base_url=BASE_URL,
        api_key=API_KEY,
        session_id=SANDBOX_SESSION_ID,
    )

tools = [sandbox_shell]

deep_agent = create_deep_agent(
    model=llm,
    backend=backend_factory,
    tools=tools,
)

In [23]:
import uuid
import re

def _extract_promise(content: str) -> str:
    """
    Replicates the Bash/Perl logic: 
    perl -0777 -pe 's/.*?<promise>(.*?)<\/promise>.*/$1/s; s/^\s+|\s+$//g; s/\s+/ /g'
    
    1. Finds content inside first <promise> tag (dotall/multiline).
    2. Trims leading/trailing whitespace.
    3. Normalizes internal whitespace to single spaces.
    """
    if not content:
        return ""
    
    # Regex for <promise>...</promise> (non-greedy, dotall)
    match = re.search(r"<promise>(.*?)</promise>", content, re.DOTALL | re.IGNORECASE)
    if not match:
        return ""
        
    raw_promise = match.group(1)
    
    # Normalize whitespace (strip ends, replace internal whitespace with single space)
    clean_promise = " ".join(raw_promise.split())
    return clean_promise

def _last_content(result):
    """Safe content extractor (same as before)."""
    messages = result.get("messages", []) if isinstance(result, dict) else []
    if not messages:
        return str(result)
    last = messages[-1]
    if hasattr(last, "content"):
        return last.content
    if isinstance(last, dict):
        return last.get("content", str(last))
    return str(last)

def run_ralph_loop(task: str, success_phrase: str, max_iterations: int = 5):
    results = []
    iteration = 1
    
    # Clean the user's success phrase to ensure fair comparison
    target_promise = " ".join(success_phrase.split())
    
    print(f"🎯 Objective: {task}")
    print(f"🔒 Success Condition: Agent must output <promise>{target_promise}</promise>")

    while iteration <= max_iterations:
        # 1. AMNESIA: New thread_id every time
        thread_id = uuid.uuid4().hex
        
        run_config = {
            "thread_id": thread_id,
            "configurable": {"sandbox_session_id": SANDBOX_SESSION_ID},
        }
        
        # 2. MATCHING THE BASH PROMPT STRATEGY
        # The Bash hook feeds the *original* prompt back but adds a system header.
        # We construct a prompt that looks like the "System Message" + "Reason" from the hook.
        
        system_header = f"🔄 Ralph iteration {iteration}"
        if max_iterations > 0:
            system_header += f"/{max_iterations}"
            
        instructions = f"""{system_header}
        
        Your previous work is in the filesystem (/workspace/). 
        Check what exists and keep building.
        
        TASK:
        {task}
        
        COMPLETION PROMISE:
        When you are truly done, you must output exactly:
        <promise>{target_promise}</promise>
        
        If you do not output this tag, the loop will continue.
        """

        print(f"\n--- {system_header} ---")
        
        # 3. EXECUTE
        result = deep_agent.invoke(
            {"messages": [{"role": "user", "content": instructions}]},
            config=run_config,
        )
        results.append(result)
        
        # 4. STRICT HOOK LOGIC
        content = _last_content(result)
        print(f"🤖 Agent Output (Snippet):\n{content[:200]}...\n")
        
        # Parse for the tag specifically (mirroring the Bash script)
        found_promise = _extract_promise(content)
        
        if found_promise:
            print(f"🔎 Found promise tag: '{found_promise}'")
            
            # Exact string comparison (bash: if [[ "$PROMISE_TEXT" = "$COMPLETION_PROMISE" ]])
            if found_promise == target_promise:
                print(f"✅ Verified <promise>{target_promise}</promise>. Task Complete!")
                break
            else:
                print(f"⚠️  Promise mismatch! Expected '{target_promise}', got '{found_promise}'. Looping...")
        else:
            if iteration >= max_iterations:
                 print(f"🛑 Max iterations ({max_iterations}) reached. Stopping.")
            else:
                 print("❌ No <promise> tag detected. Re-looping...")
            
        iteration += 1
        
    return results

In [25]:
TASK = "Build a tiny python checklist app in /workspace and keep iterating on it."
SUCCESS_PHRASE = "ALL_DONE_AND_FUNCTIONAL"
MAX_ITERATIONS = 5

results = run_ralph_loop(TASK, SUCCESS_PHRASE, MAX_ITERATIONS)

🎯 Objective: Build a tiny python checklist app in /workspace and keep iterating on it.
🔒 Success Condition: Agent must output <promise>ALL_DONE_AND_FUNCTIONAL</promise>

--- 🔄 Ralph iteration 1/5 ---
🤖 Agent Output (Snippet):
<promise>ALL_DONE_AND_FUNCTIONAL</promise>...

🔎 Found promise tag: 'ALL_DONE_AND_FUNCTIONAL'
✅ Verified <promise>ALL_DONE_AND_FUNCTIONAL</promise>. Task Complete!


In [26]:
flat_messages = [msg for item in results for msg in item['messages']]

for m in flat_messages:
    print(m.pretty_repr(html=True))

================================ Human Message =================================

🔄 Ralph iteration 1/5

        Your previous work is in the filesystem (/workspace/). 
        Check what exists and keep building.

        TASK:
        Build a tiny python checklist app in /workspace and keep iterating on it.

        COMPLETION PROMISE:
        When you are truly done, you must output exactly:
        <promise>ALL_DONE_AND_FUNCTIONAL</promise>

        If you do not output this tag, the loop will continue.
        
================================== Ai Message ==================================
Tool Calls:
  ls (430d57a8-2e25-4c78-8820-4880f6dc3b01)
 Call ID: 430d57a8-2e25-4c78-8820-4880f6dc3b01
  Args:
    path: /workspace/
================================= Tool Message =================================
Name: ls

['/workspace/.', '/workspace/..', '/workspace/checklist']
================================== Ai Message ==================================
Tool Calls:
  ls (4457b8d1-2ffa-4a

In [27]:
# Optional: stop the sandbox session when you are done
client.stop_sandbox_session(SANDBOX_SESSION_ID)